# ex1124

Google Colab と研究室のGPGPUマシンの性能比較のためのほげ

## 初期設定

In [1]:
from google.colab import drive
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [2]:
pathGDrive = 'gdrive/My Drive/191124-vsColab'
!ls 'gdrive/My Drive/191124-vsColab'

data191124.py	  ex191124.ipynb	  mnist
ex191124_cnn.py   ex191124_mlp.py	  mnist.py
ex191124_data.py  ex191124-params.pickle  __pycache__


In [0]:
import numpy as np
import cv2
import datetime
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
### GoogleDrive上の hoge.py を import できるように
#
sys.path.append(pathGDrive)
import ex191124_data as data
#import ex191124_mlp as network
import ex191124_cnn as network

In [5]:
### device
#
use_gpu_if_available = False
if use_gpu_if_available and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('# using', device)

# using cpu


In [6]:
### loading and setting the data

#datL, labL, datT, labT = data.loadMNISTData(pathMNIST=os.path.join(pathGDrive, 'mnist'))
datL, labL, datT, labT = data.loadMNISTData(pathMNIST=os.path.join(pathGDrive, 'mnist'), forCNN=True)
NL = datL.shape[0]
NT = datT.shape[0]
K = 10
batchsize = 128

XL = torch.from_numpy(datL.astype(np.float32)).to(device)
YL = torch.from_numpy(labL).to(device)

bindexL = data.makeBatchIndex(NL, batchsize)
nbatchL = bindexL.shape[0]

print(datL.shape, XL.shape, YL.shape)

XT = torch.from_numpy(datT.astype(np.float32)).to(device)
YT = torch.from_numpy(labT).to(device)

bindexT = data.makeBatchIndex(NT, batchsize)
nbatchT = bindexT.shape[0]

print(datT.shape, XT.shape, YT.shape)

(60000, 1, 28, 28) (60000,)
(10000, 1, 28, 28) (10000,)
(60000, 1, 28, 28) torch.Size([60000, 1, 28, 28]) torch.Size([60000])
(10000, 1, 28, 28) torch.Size([10000, 1, 28, 28]) torch.Size([10000])


## 学習

In [0]:
# うっかり全てのセルを実行とかしたときのためのストッパー
hoge

NameError: ignored

In [7]:
### initializing the network
#
torch.manual_seed(0)
#net = network.MLP(XL.shape[1], 1000, 1000, K)
net = network.CNN(XL.shape[1:], 16, 32, 1000, K)
model = net.to(device)
print(net)
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(optimizer)

# input: torch.Size([1, 1, 28, 28])
# conv1 output: torch.Size([1, 16, 13, 13])
# conv2 output: torch.Size([1, 32, 5, 5])
# flatten output: torch.Size([1, 800])
# fc output: torch.Size([1, 1000])
# softmax output: torch.Size([1, 10])
CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=800, out_features=1000, bias=True)
    (1): ReLU()
  )
  (softmax): Sequential(
    (0): Linear(in_features=1000, out_features=10, bias=True)
    (1): LogSoftmax()
  )
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [0]:
### training
#
nitr = 10000
nd = 0
start = datetime.datetime.now()

for i in range(nitr):

    if (i < 500 and i % 100 == 0) or (i % 500 == 0):
        model.eval()
        loss, ncorrect = network.evaluate(model, XL, YL, bindexL)
        print(i, nd, loss/NL, ncorrect/NL)

    model.train()
    ib = np.random.randint(0, nbatchL)
    ii = np.where(bindexL[ib, :])[0]
    optimizer.zero_grad()
    output = model(XL[ii, ::])
    loss = F.nll_loss(output, YL[ii])
    loss.backward()
    optimizer.step()

    nd += ii.shape[0]

model.eval()
loss, ncorrect = network.evaluate(model, XL, YL, bindexL)
print(nitr, nd, loss/NL, ncorrect/NL)

print('# elapsed time: ', datetime.datetime.now() - start)
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 0 14.082742146809895 0.09838333333333334
100 12800 0.16392775155703226 0.94875
200 25600 0.11297127035856247 0.966
300 38400 0.07848469469745954 0.9751
400 51200 0.08052248554825783 0.9749666666666666
500 63968 0.058747948459287484 0.9814
1000 127968 0.03005110102519393 0.9904666666666667
1500 191968 0.022755657086024682 0.9930166666666667
2000 255936 0.02382636903449893 0.9926333333333334
2500 319872 0.02517553679080059 0.9918833333333333
3000 383808 0.014569507723402542 0.9954666666666667
3500 447776 0.02457671113392959 0.9925666666666667
4000 511776 0.023184303542335208 0.9926166666666667
4500 575744 0.02073916367557831 0.9940833333333333
5000 639744 0.016220717712600405 0.9949833333333333
5500 703680 0.013728965956260799 0.9956833333333334
6000 767648 0.015792669472155588 0.9959
6500 831552 0.011035342351863438 0.9970166666666667
7000 895520 0.013382159414924535 0.9963333333333333
7500 959520 0.01451202318450584 0.99615
8000 1023488 0.014456832812016485 0.9959666666666667
8500 10

In [9]:
### saving the model
#
fnModel = pathGDrive + '/ex191124-params.pickle'
with open(fnModel, mode = 'wb') as f:
    torch.save(model.state_dict(), f)
    print('# The model is saved to ', fnModel)

# The model is saved to  gdrive/My Drive/191124-vsColab/ex191124-params.pickle


## テスト

In [10]:
fnModel = pathGDrive + '/ex191124-params.pickle'

### loading the network
#
torch.manual_seed(0)
#net = network.MLP(XT.shape[1], 1000, 1000, K)
net = network.CNN(XL.shape[1:], 16, 32, 1000, K)
with open(fnModel, mode = 'rb') as f:
    net.load_state_dict(torch.load(f))
    model = net.to(device)
print(net)

# input: torch.Size([1, 1, 28, 28])
# conv1 output: torch.Size([1, 8, 13, 13])
# conv2 output: torch.Size([1, 8, 5, 5])
# flatten output: torch.Size([1, 200])
# fc output: torch.Size([1, 1000])
# softmax output: torch.Size([1, 10])
CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=200, out_features=1000, bias=True)
    (1): ReLU()
  )
  (softmax): Sequential(
    (0): Linear(in_features=1000, out_features=10, bias=True)
    (1): LogSoftmax()
  )
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
model.eval()
loss, ncorrect = network.evaluate(model, XT, YT, bindexT)
print(loss/NT, ncorrect/NT)

0.11454798817634583 0.9862


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
